# Option Greeks — Overview

The **Greeks** are sensitivity measures that describe how an option's price changes with respect to various factors. They are essential for:
- **Hedging** — managing and neutralizing risk exposures
- **Risk management** — understanding P&L sensitivities
- **Trading** — identifying profitable opportunities

---

## The Five Primary Greeks

| Greek | Symbol | Measures Sensitivity To | Range |
|-------|--------|-------------------------|-------|
| **Delta** | Δ | Underlying price | Call: [0,1], Put: [-1,0] |
| **Gamma** | Γ | Rate of change of Delta | Always ≥ 0 |
| **Theta** | Θ | Time decay | Usually negative |
| **Vega** | ν | Implied volatility | Always ≥ 0 |
| **Rho** | ρ | Interest rates | Positive for calls, negative for puts |

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import norm

def black_scholes_greeks(S, K, T, r, sigma, option_type='call'):
    """Calculate Black-Scholes price and all Greeks."""
    if T <= 0:
        return {'price': max(S-K, 0) if option_type=='call' else max(K-S, 0),
                'delta': 1 if S>K else 0, 'gamma': 0, 'theta': 0, 'vega': 0, 'rho': 0}
    
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    if option_type == 'call':
        price = S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
        delta = norm.cdf(d1)
        rho = K*T*np.exp(-r*T)*norm.cdf(d2) / 100
    else:
        price = K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)
        delta = norm.cdf(d1) - 1
        rho = -K*T*np.exp(-r*T)*norm.cdf(-d2) / 100
    
    gamma = norm.pdf(d1) / (S*sigma*np.sqrt(T))
    theta = (-S*norm.pdf(d1)*sigma/(2*np.sqrt(T)) - r*K*np.exp(-r*T)*norm.cdf(d2 if option_type=='call' else -d2)) / 365
    vega = S*norm.pdf(d1)*np.sqrt(T) / 100
    
    return {'price': price, 'delta': delta, 'gamma': gamma, 'theta': theta, 'vega': vega, 'rho': rho}

## 1. Delta (Δ) — Price Sensitivity

$$\Delta = \frac{\partial V}{\partial S}$$

**Interpretation:**
- How much the option price changes for a $1 move in the underlying
- Also approximates the probability of finishing in-the-money
- Used to calculate hedge ratio (delta-neutral hedging)

**Delta Behavior:**
- ATM options: Δ ≈ 0.50 (calls) or -0.50 (puts)
- Deep ITM: Δ → 1 (calls) or -1 (puts)
- Deep OTM: Δ → 0

In [ ]:
# Delta across different spot prices and times to expiry
S_range = np.linspace(70, 130, 100)
K, r, sigma = 100, 0.05, 0.20

fig = make_subplots(rows=1, cols=2, subplot_titles=('Call Delta', 'Put Delta'))

for T, color in [(0.25, 'blue'), (0.5, 'green'), (1.0, 'red')]:
    deltas_call = [black_scholes_greeks(S, K, T, r, sigma, 'call')['delta'] for S in S_range]
    deltas_put = [black_scholes_greeks(S, K, T, r, sigma, 'put')['delta'] for S in S_range]
    
    fig.add_trace(go.Scatter(x=S_range, y=deltas_call, name=f'T={T}y', line=dict(color=color)), row=1, col=1)
    fig.add_trace(go.Scatter(x=S_range, y=deltas_put, name=f'T={T}y', line=dict(color=color), showlegend=False), row=1, col=2)

fig.add_hline(y=0.5, line_dash='dash', line_color='gray', row=1, col=1)
fig.add_hline(y=-0.5, line_dash='dash', line_color='gray', row=1, col=2)
fig.add_vline(x=K, line_dash='dot', line_color='black')

fig.update_layout(title='Delta vs Spot Price (K=100, σ=20%)', template='plotly_white')
fig.update_xaxes(title_text='Spot Price')
fig.update_yaxes(title_text='Delta')
fig

## 2. Gamma (Γ) — Delta Sensitivity

$$\Gamma = \frac{\partial^2 V}{\partial S^2} = \frac{\partial \Delta}{\partial S}$$

**Interpretation:**
- Rate of change of Delta as the underlying moves
- Measures the convexity of the option payoff
- High Gamma = Delta changes rapidly → more frequent rehedging needed

**Gamma Behavior:**
- Highest for ATM options near expiration
- Long options: always positive Gamma (good — convexity works for you)
- Short options: negative Gamma exposure (dangerous near expiry)

In [ ]:
# Gamma surface: shows how Gamma peaks at ATM and near expiry
S_range = np.linspace(80, 120, 50)
T_range = np.linspace(0.02, 1.0, 50)
S_grid, T_grid = np.meshgrid(S_range, T_range)

gamma_grid = np.array([[black_scholes_greeks(S, K, T, r, sigma, 'call')['gamma'] 
                        for S in S_range] for T in T_range])

fig = go.Figure(data=[go.Surface(x=S_grid, y=T_grid, z=gamma_grid, colorscale='Viridis')])
fig.update_layout(
    title='Gamma Surface (K=100, σ=20%)',
    scene=dict(xaxis_title='Spot Price', yaxis_title='Time to Expiry', zaxis_title='Gamma'),
    template='plotly_white'
)
fig

## 3. Theta (Θ) — Time Decay

$$\Theta = \frac{\partial V}{\partial t}$$

**Interpretation:**
- How much value the option loses each day (all else equal)
- Usually negative for long options (time works against you)
- Theta accelerates as expiration approaches

**Theta Behavior:**
- Highest (most negative) for ATM options near expiry
- Deep ITM/OTM options have less time value to lose
- Option sellers collect Theta; option buyers pay it

In [ ]:
# Theta decay over time
days_to_expiry = np.arange(1, 91)
T_values = days_to_expiry / 365

fig = go.Figure()
for moneyness, S in [('ITM', 110), ('ATM', 100), ('OTM', 90)]:
    thetas = [black_scholes_greeks(S, K, T, r, sigma, 'call')['theta'] for T in T_values]
    prices = [black_scholes_greeks(S, K, T, r, sigma, 'call')['price'] for T in T_values]
    fig.add_trace(go.Scatter(x=days_to_expiry, y=thetas, name=f'{moneyness} (S={S})'))

fig.update_layout(
    title='Theta (Daily Decay) vs Days to Expiry',
    xaxis_title='Days to Expiry',
    yaxis_title='Theta ($/day)',
    template='plotly_white'
)
fig

## 4. Vega (ν) — Volatility Sensitivity

$$\nu = \frac{\partial V}{\partial \sigma}$$

**Interpretation:**
- How much the option price changes for a 1% change in implied volatility
- Always positive for long options (higher vol = more valuable)
- Critical for volatility trading strategies

**Vega Behavior:**
- Highest for ATM options with longer time to expiry
- Near-expiry options have low Vega (vol matters less)
- Long straddles/strangles are long Vega

In [ ]:
# Vega across moneyness and time
fig = make_subplots(rows=1, cols=2, subplot_titles=('Vega vs Spot', 'Vega vs Time'))

# Vega vs Spot for different maturities
for T, color in [(0.25, 'blue'), (0.5, 'green'), (1.0, 'red')]:
    vegas = [black_scholes_greeks(S, K, T, r, sigma, 'call')['vega'] for S in S_range]
    fig.add_trace(go.Scatter(x=S_range, y=vegas, name=f'T={T}y', line=dict(color=color)), row=1, col=1)

# Vega vs Time for different moneyness
T_range = np.linspace(0.05, 2.0, 100)
for moneyness, S, color in [('ITM', 110, 'green'), ('ATM', 100, 'blue'), ('OTM', 90, 'red')]:
    vegas = [black_scholes_greeks(S, K, T, r, sigma, 'call')['vega'] for T in T_range]
    fig.add_trace(go.Scatter(x=T_range, y=vegas, name=moneyness, line=dict(color=color)), row=1, col=2)

fig.update_layout(title='Vega Sensitivity', template='plotly_white')
fig.update_xaxes(title_text='Spot Price', row=1, col=1)
fig.update_xaxes(title_text='Time to Expiry (years)', row=1, col=2)
fig.update_yaxes(title_text='Vega (per 1% vol)')
fig

## 5. Rho (ρ) — Interest Rate Sensitivity

$$\rho = \frac{\partial V}{\partial r}$$

**Interpretation:**
- How much the option price changes for a 1% change in interest rates
- Positive for calls (higher rates → higher call value)
- Negative for puts (higher rates → lower put value)

**Rho Behavior:**
- Generally the least important Greek for short-dated options
- More significant for long-dated options (LEAPS)
- Matters more in high interest rate environments

## Greek Interactions and Trading Applications

### Key Relationships

| Position | Delta | Gamma | Theta | Vega |
|----------|-------|-------|-------|------|
| Long Call | + | + | − | + |
| Long Put | − | + | − | + |
| Short Call | − | − | + | − |
| Short Put | + | − | + | − |
| Long Straddle | 0 | + | − | + |
| Short Straddle | 0 | − | + | − |

### Hedging Strategies

1. **Delta Hedging**: Buy/sell underlying to neutralize Delta
2. **Gamma Scalping**: Profit from Gamma by rehedging frequently
3. **Vega Hedging**: Use other options to offset volatility exposure

### P&L Attribution

$$\Delta P\&L \approx \Delta \cdot dS + \frac{1}{2}\Gamma \cdot dS^2 + \Theta \cdot dt + \nu \cdot d\sigma$$

In [ ]:
# All Greeks summary for a sample option
S, K, T, r, sigma = 100, 100, 0.5, 0.05, 0.20

call_greeks = black_scholes_greeks(S, K, T, r, sigma, 'call')
put_greeks = black_scholes_greeks(S, K, T, r, sigma, 'put')

print(f"ATM Option (S={S}, K={K}, T={T}y, r={r*100}%, σ={sigma*100}%)")
print("="*60)
print(f"{'Greek':<10} {'Call':>12} {'Put':>12}")
print("-"*36)
for greek in ['price', 'delta', 'gamma', 'theta', 'vega', 'rho']:
    print(f"{greek.capitalize():<10} {call_greeks[greek]:>12.4f} {put_greeks[greek]:>12.4f}")

## Quick Reference

| Greek | Formula (Call) | What It Tells You |
|-------|----------------|-------------------|
| **Delta** | $N(d_1)$ | Price move per $1 underlying move |
| **Gamma** | $\frac{N'(d_1)}{S\sigma\sqrt{T}}$ | Acceleration of Delta |
| **Theta** | $-\frac{S N'(d_1)\sigma}{2\sqrt{T}} - rKe^{-rT}N(d_2)$ | Daily time decay |
| **Vega** | $S\sqrt{T}N'(d_1)$ | Price move per 1% vol change |
| **Rho** | $KTe^{-rT}N(d_2)$ | Price move per 1% rate change |

Where:
- $d_1 = \frac{\ln(S/K) + (r + \sigma^2/2)T}{\sigma\sqrt{T}}$
- $d_2 = d_1 - \sigma\sqrt{T}$
- $N(x)$ = standard normal CDF
- $N'(x)$ = standard normal PDF